# Snowprofile: Read, Inspect, Plot, and Export CAAML 6

This tutorial teaches avalanche practitioners how to use the `snowprofile` Python package with CAAML v6 files in Google Colab. You'll learn to read snow profile data, explore the SnowProfile object structure, analyze profiles, create visualizations, and export back to CAAML format.

**Demo File**: This tutorial uses `snowprofiles/example_profile.caaml` as the example file. When running in Colab, the notebook will automatically download it from the repository. When running locally from the repository, it will use the file directly.

## Goals and Workflow

1. **Install** → Install `snowprofile` and dependencies
2. **Read CAAML** → Load a user-provided CAAML v6 file (or use the example file)
3. **Explore** → Inspect the SnowProfile object structure and data
4. **Analyze** → Compute temperature gradients and examine stratigraphy
5. **Visualize** → Create plots using built-in plotting functions
6. **Export** → Write profiles back to CAAML format

## Units and Conventions

- **Heights**: Measured in meters (SI units)
- **Vertical coordinate**: Zero is at the bottom of the profile
- **Unit conversions**: The `snowprofile` package handles unit conversions automatically on read/write operations
- **CAAML versions**: Reader supports up to CAAML 6.0.5; writer supports 6.0.5 (default) and 6.0.6

## References

- Package: `pip install snowprofile`
- CAAML I/O: `snowprofile.io.read_caaml6_xml`, `snowprofile.io.write_caaml6_xml`
- Plotting: `snowprofile.plot.plot_simple`, `snowprofile.plot.plot_full`, `snowprofile.plot.plot_utils`



In [1]:
# Environment checks
import sys

# Confirm we're in Colab
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    print("✓ Running in Google Colab")
else:
    print("⚠ Not running in Colab - some features may not work")

# Set matplotlib inline and silence warnings
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for inline plotting (Colab handles this automatically)
import matplotlib
if IN_COLAB:
    matplotlib.use('inline')  # Colab uses inline by default
else:
    matplotlib.use('Agg')  # Non-interactive backend for local testing
print(f"✓ Matplotlib backend: {matplotlib.get_backend()}")

# Enable inline plotting
%matplotlib inline



⚠ Not running in Colab - some features may not work
✓ Matplotlib backend: Agg


In [2]:
# Install snowprofile and dependencies
%pip install -q snowprofile pandas numpy matplotlib rich

# Verify installation
import snowprofile
print(f"✓ snowprofile version: {snowprofile.__version__}")



Note: you may need to restart the kernel to use updated packages.


AttributeError: module 'snowprofile' has no attribute '__version__'

In [ ]:
# Imports
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

# snowprofile imports
import snowprofile
from snowprofile import io as spio
from snowprofile import plot as spplot
from snowprofile.plot import plot_utils as sppu

# Optional rich printing with fallback
try:
    from rich import print as rprint
except ImportError:
    rprint = print

print("✓ All imports successful")



In [ ]:
# Data intake: Upload CAAML file or use example file

# Option A: Use example file from repository (for tutorial/demo)
# If running from the repository, use the example file
EXAMPLE_FILE = Path("snowprofiles/example_profile.caaml")

if EXAMPLE_FILE.exists():
    CAAML_PATH = EXAMPLE_FILE
    print(f"✓ Using example file: {CAAML_PATH}")
    print(f"  This is a demo CAAML v6 file from the repository")
elif IN_COLAB:
    # Option B: Download example file from repository (Colab)
    print("Downloading example CAAML file from repository...")
    import urllib.request
    example_url = "https://raw.githubusercontent.com/Austfi/SNOWPACKforPatrollers/main/snowprofiles/example_profile.caaml"
    CAAML_PATH = Path("example_profile.caaml")
    try:
        urllib.request.urlretrieve(example_url, str(CAAML_PATH))
        print(f"✓ Downloaded example file: {CAAML_PATH}")
    except Exception as e:
        print(f"⚠ Could not download example file: {e}")
        print("Falling back to file upload...")
        # Option C: Colab file upload (fallback)
        from google.colab import files
        print("Upload a CAAML v6 file (.caaml or .xml):")
        uploaded = files.upload()
        CAAML_PATH = Path(next(iter(uploaded)))
        print(f"✓ Uploaded: {CAAML_PATH}")
else:
    # Option D: Local testing - set path here
    CAAML_PATH = Path("example.caaml")  # Modify as needed
    print(f"⚠ Using local path: {CAAML_PATH}")
    print(f"  For demo, ensure snowprofiles/example_profile.caaml exists")

# Verify file exists
assert CAAML_PATH.exists(), f"File not found: {CAAML_PATH}"
print(f"✓ File exists: {CAAML_PATH.absolute()}")





In [ ]:
# CAAML sanity check - Light XML parsing before using snowprofile

tree = ET.parse(CAAML_PATH)
root = tree.getroot()

# Detect CAAML v6 namespace
CAAML_NS_URIS = [
    "http://caaml.org/Schemas/SnowProfileIACS/v6.0.3",
    "http://caaml.org/Schemas/SnowProfileIACS/v6.0.4",
    "http://caaml.org/Schemas/SnowProfileIACS/v6.0.5",
    "http://caaml.org/Schemas/SnowProfileIACS/v6.0.6"
]

# Find namespace (common prefixes)
ns_uri = None
for uri in CAAML_NS_URIS:
    # Try common namespace prefixes
    for prefix in ['', '{', 'caaml:', '{caaml:']:
        test_ns = f"{{{uri}}}" if prefix == '{' else uri
        if test_ns in root.tag or uri in root.tag:
            ns_uri = uri
            break
    if ns_uri:
        break

if not ns_uri:
    # Try to extract from root tag
    if '}' in root.tag:
        ns_uri = root.tag.split('}')[0][1:]
    else:
        raise ValueError("Could not detect CAAML v6 namespace. Is this a CAAML v6 file?")

print(f"✓ CAAML namespace detected: {ns_uri}")

# Count layers and temperature observations
ns = {"caaml": ns_uri}
layer_count = len(root.findall(".//caaml:Layer", ns))
temp_obs_count = len(root.findall(".//caaml:tempProfile", ns))

print(f"✓ Layers found: {layer_count}")
print(f"✓ Temperature profiles found: {temp_obs_count}")

# Extract timePosition if present
time_elem = root.find(".//caaml:timePosition", ns)
if time_elem is not None:
    time_str = time_elem.text or time_elem.get('value', 'N/A')
    print(f"✓ Time position: {time_str}")
else:
    print("⚠ No timePosition found")

# Gather all loc attribute values
loc_values = []
for elem in root.iter():
    if "loc" in elem.attrib:
        loc_values.append(elem.attrib["loc"])

if loc_values:
    from collections import Counter
    loc_freq = Counter(loc_values)
    print(f"\n✓ Loc attribute frequency:")
    for loc, count in loc_freq.most_common():
        print(f"  {loc}: {count}")
    
    # Check for non-standard tags
    standard = {"T", "M", "B"}
    non_standard = set(loc_freq.keys()) - standard
    if non_standard:
        print(f"\n⚠ Non-standard loc values found: {non_standard}")
        print("  These may cause validation errors. The library validates tables strictly.")
        print("  Standard values are: T (top), M (middle), B (bottom)")
else:
    print("⚠ No loc attributes found")



In [ ]:
# Helper: sanitize_caaml_loc
# This function normalizes non-standard loc attribute values to {T, M, B} or removes them

def sanitize_caaml_loc(in_path: Path, out_path: Path, ns_uri="http://caaml.org/Schemas/SnowProfileIACS/v6.0.3"):
    """
    Sanitize CAAML file by normalizing loc attributes to standard values {T, M, B}.
    
    Some exporters use loc="Surface", loc="Bottom", etc. This function:
    - Maps common variants to standard values (Surface/Top→T, Bottom→B, Mid/Middle→M)
    - Removes unknown loc values to avoid validation errors
    """
    ns = "{"+ns_uri+"}"
    tree = ET.parse(in_path)
    root = tree.getroot()
    
    mapping = {
        "S": "T", "Surface": "T", "Top": "T", "top": "T",
        "P": "B", "Bottom": "B", "Btm": "B", "bottom": "B",
        "L": "M", "Mid": "M", "Middle": "M", "middle": "M"
    }
    
    for elem in root.iter():
        if "loc" in elem.attrib:
            v = elem.attrib["loc"]
            if v in {"T","M","B"}: 
                continue
            if v in mapping:
                elem.set("loc", mapping[v])
            else:
                del elem.attrib["loc"]
    
    tree.write(out_path, encoding="utf-8", xml_declaration=True)
    print(f"✓ Sanitized file written to: {out_path}")



In [ ]:
# Read CAAML file

try:
    sp = spio.read_caaml6_xml(str(CAAML_PATH))
    print(f"✓ Successfully read CAAML file: {CAAML_PATH.name}")
except ValueError as e:
    if "Unauthorized value for key loc" in str(e) or "validation" in str(e).lower():
        print(f"⚠ Validation error detected: {e}")
        print("Attempting to sanitize loc attributes...")
        
        # Create sanitized copy
        fixed_path = CAAML_PATH.parent / f"{CAAML_PATH.stem}.fixed{CAAML_PATH.suffix}"
        sanitize_caaml_loc(CAAML_PATH, fixed_path, ns_uri)
        
        # Retry reading
        try:
            sp = spio.read_caaml6_xml(str(fixed_path))
            print(f"✓ Successfully read sanitized file: {fixed_path.name}")
            CAAML_PATH = fixed_path  # Update path for reference
        except Exception as e2:
            raise ValueError(f"Failed to read even after sanitization: {e2}")
    else:
        raise

# Verify we have a valid SnowProfile object
assert sp is not None, "Failed to read SnowProfile object"
print(f"✓ SnowProfile object created successfully")



## SnowProfile Summary

The SnowProfile object contains time, location, depth/SWE, stratigraphy, and various profile data (temperature, density, hardness, etc.). Let's explore its structure.


In [ ]:
# SnowProfile summary

print("=" * 60)
print("SNOWPROFILE SUMMARY")
print("=" * 60)

# Time information
print(f"\nTime:")
if hasattr(sp, 'record_time') and sp.record_time:
    print(f"  Record time: {sp.record_time}")
if hasattr(sp, 'report_time') and sp.report_time:
    print(f"  Report time: {sp.report_time}")

# Location information
print(f"\nLocation:")
if hasattr(sp, 'name') and sp.name:
    print(f"  Name: {sp.name}")
if hasattr(sp, 'elevation') and sp.elevation is not None:
    print(f"  Elevation: {sp.elevation} m")
if hasattr(sp, 'aspect') and sp.aspect is not None:
    print(f"  Aspect: {sp.aspect}°")
if hasattr(sp, 'slope') and sp.slope is not None:
    print(f"  Slope: {sp.slope}°")
if hasattr(sp, 'latitude') and sp.latitude is not None:
    print(f"  Latitude: {sp.latitude}°")
if hasattr(sp, 'longitude') and sp.longitude is not None:
    print(f"  Longitude: {sp.longitude}°")

# Profile dimensions
print(f"\nProfile dimensions:")
if hasattr(sp, 'profile_depth') and sp.profile_depth is not None:
    print(f"  Profile depth: {sp.profile_depth:.2f} m")
if hasattr(sp, 'profile_swe') and sp.profile_swe is not None:
    print(f"  Profile SWE: {sp.profile_swe:.1f} mm")

# Count available profiles
print(f"\nAvailable profiles:")
profile_types = {
    'temperature_profiles': getattr(sp, 'temperature_profiles', None),
    'density_profiles': getattr(sp, 'density_profiles', None),
    'hardness_profiles': getattr(sp, 'hardness_profiles', None),
    'ssa_profiles': getattr(sp, 'ssa_profiles', None),
    'strength_profiles': getattr(sp, 'strength_profiles', None),
    'lwc_profiles': getattr(sp, 'lwc_profiles', None),
    'other_scalar_profiles': getattr(sp, 'other_scalar_profiles', None),
    'other_vectorial_profiles': getattr(sp, 'other_vectorial_profiles', None),
}

for name, profile_list in profile_types.items():
    if profile_list is not None:
        count = len(profile_list) if isinstance(profile_list, list) else (1 if profile_list else 0)
        print(f"  {name}: {count}")
    else:
        print(f"  {name}: not available")

print("=" * 60)



In [ ]:
# Stratigraphy inspection

if hasattr(sp, 'stratigraphy_profile') and sp.stratigraphy_profile is not None:
    print("✓ Stratigraphy profile available")
    
    # Get stratigraphy data
    strat_data = sp.stratigraphy_profile
    if hasattr(strat_data, 'data'):
        df = strat_data.data
    elif hasattr(strat_data, 'data_dict'):
        import pandas as pd
        df = pd.DataFrame(strat_data.data_dict)
    else:
        df = None
    
    if df is not None and not df.empty:
        print(f"\nStratigraphy DataFrame shape: {df.shape}")
        print(f"\nFirst few layers:")
        print(df.head())
        
        # Check for expected columns
        expected_cols = ['top_height', 'bottom_height', 'thickness']
        has_expected = all(col in df.columns for col in expected_cols)
        assert has_expected, f"Missing expected columns. Found: {list(df.columns)}"
        
        # Map hand-hardness strings to numeric for quick stats
        hardness_map = {'F': 1, '4F': 2, '1F': 3, 'P': 4, 'K': 5}
        if 'hardness' in df.columns:
            df['hardness_numeric'] = df['hardness'].map(hardness_map)
            print(f"\nHand-hardness mapping (conventional indices):")
            for k, v in hardness_map.items():
                print(f"  {k} → {v}")
        
        # Sanity check: compare profile_depth vs max layer bottom
        if 'bottom_height' in df.columns:
            max_bottom = df['bottom_height'].max()
            print(f"\n✓ Max layer bottom: {max_bottom:.2f} m")
            if hasattr(sp, 'profile_depth') and sp.profile_depth:
                print(f"✓ Profile depth: {sp.profile_depth:.2f} m")
                diff = abs(max_bottom - sp.profile_depth)
                if diff > 0.01:
                    print(f"⚠ Difference: {diff:.2f} m")
        
        # Histogram of layer thickness
        if 'thickness' in df.columns:
            import matplotlib.pyplot as plt
            plt.figure(figsize=(8, 4))
            plt.hist(df['thickness'], bins=20, edgecolor='black')
            plt.xlabel('Layer thickness (m)')
            plt.ylabel('Frequency')
            plt.title('Distribution of Layer Thickness')
            plt.grid(True, alpha=0.3)
            plt.show()
    else:
        print("⚠ Stratigraphy data not available as DataFrame")
else:
    print("⚠ No stratigraphy profile available")



## Temperature Gradient Analysis

The temperature gradient indicates how quickly temperature changes with depth. Strong gradients can indicate instability or rapid temperature changes.

### Mathematical Formula

The temperature gradient is computed using a centered finite-difference approximation:

$$\frac{dT}{dz}\Big|_{z_i} \approx \frac{T_{i+1} - T_{i-1}}{z_{i+1} - z_{i-1}}$$

**Units**: °C/m (degrees Celsius per meter)

**Context**: 
- Strong gradients (typically ≥ 10 °C/m) can indicate:
  - Rapid temperature changes
  - Potential instability
  - Temperature inversions
  
**Note**: Interpretation depends on context - strong gradients near the surface during cold snaps are different from mid-pack gradients.


In [ ]:
# Temperature gradient analysis

if hasattr(sp, 'temperature_profiles') and sp.temperature_profiles:
    temp_profiles = sp.temperature_profiles
    if len(temp_profiles) > 0:
        # Use first temperature profile
        temp_profile = temp_profiles[0]
        
        # Get data
        if hasattr(temp_profile, 'data'):
            temp_data = temp_profile.data
        elif hasattr(temp_profile, 'data_dict'):
            temp_data = pd.DataFrame(temp_profile.data_dict)
        else:
            temp_data = None
        
        if temp_data is not None and not temp_data.empty:
            # Ensure we have height and temperature columns
            if 'height' in temp_data.columns and 'temperature' in temp_data.columns:
                # Sort by height
                temp_data = temp_data.sort_values('height')
                heights = temp_data['height'].values
                temps = temp_data['temperature'].values
                
                # Remove NaN values
                valid_mask = ~(np.isnan(heights) | np.isnan(temps))
                heights = heights[valid_mask]
                temps = temps[valid_mask]
                
                if len(heights) > 1:
                    # Compute gradient using numpy's gradient function
                    # edge_order=2 uses second-order accurate differences at boundaries
                    dTdz = np.gradient(temps, heights, edge_order=2)
                    
                    # Create strong gradient flag (≥ 10 °C/m threshold)
                    strong_gradient_flag = np.abs(dTdz) >= 10
                    
                    print(f"✓ Temperature profile analyzed")
                    print(f"  Data points: {len(heights)}")
                    print(f"  Height range: {heights.min():.2f} - {heights.max():.2f} m")
                    print(f"  Temperature range: {temps.min():.2f} - {temps.max():.2f} °C")
                    print(f"  Gradient range: {dTdz.min():.2f} - {dTdz.max():.2f} °C/m")
                    print(f"  Strong gradients (≥10 °C/m): {np.sum(strong_gradient_flag)} points")
                    
                    # Plot temperature vs height
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
                    
                    # Temperature profile
                    ax1.plot(temps, heights, 'o-', markersize=4)
                    ax1.set_xlabel('Temperature (°C)')
                    ax1.set_ylabel('Height (m)')
                    ax1.set_title('Temperature Profile')
                    ax1.grid(True, alpha=0.3)
                    ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
                    
                    # Temperature gradient
                    ax2.plot(dTdz, heights, 'o-', markersize=4, color='orange')
                    ax2.axvline(x=0, color='k', linestyle='--', alpha=0.3)
                    ax2.axvline(x=10, color='r', linestyle='--', alpha=0.5, label='±10 °C/m threshold')
                    ax2.axvline(x=-10, color='r', linestyle='--', alpha=0.5)
                    ax2.set_xlabel('Temperature Gradient (°C/m)')
                    ax2.set_ylabel('Height (m)')
                    ax2.set_title('Temperature Gradient')
                    ax2.grid(True, alpha=0.3)
                    ax2.legend()
                    
                    plt.tight_layout()
                    plt.show()
                    
                    print("\nNote: Heights are in meters, zero at bottom. The library handles unit conversions automatically.")
                else:
                    print("⚠ Insufficient data points for gradient calculation")
            else:
                print(f"⚠ Missing required columns. Available: {list(temp_data.columns)}")
        else:
            print("⚠ Temperature data not available")
    else:
        print("⚠ No temperature profiles available")
else:
    print("⚠ Temperature profile data not available")



In [ ]:
# Helper: plot_full_safe
# Robust wrapper for plot_full that handles single-Axes edge case

def plot_full_safe(sp,
                   index_temperature_profiles='all',
                   index_density_profiles='all', style_density_profiles='step',
                   index_hardness_profiles='all', style_hardness_profiles='step',
                   index_impurity_profiles='all', style_impurity_profiles='point',
                   index_ssa_profiles='all', style_ssa_profiles='point',
                   index_strength_profiles='all', style_strength_profiles='point',
                   index_lwc_profiles='all', style_lwc_profiles='step',
                   index_scalar_profiles='all',
                   **kwargs):
    """
    Safe wrapper for plot_full that handles single-Axes edge case.
    
    Creates subplots with squeeze=False and flattens via axs.ravel() to avoid
    the TypeError: 'Axes' object is not iterable when there's only one subplot.
    """
    import matplotlib.pyplot as plt
    from snowprofile.plot import plot_utils
    
    to_plot = {
        'Stratigraphy profile': {'plot_style':'profile','key':'hardness','xlabel':'Hand hardness','data':sp.stratigraphy_profile,'index':None},
        'Hardness profile': {'plot_style':style_hardness_profiles,'key':'hardness','xlabel':'Hardness (N)','data':sp.hardness_profiles,'index':index_hardness_profiles},
        'Temperature profile': {'plot_style':'point','key':'temperature','xlabel':'Temperature (°C)','data':sp.temperature_profiles,'index':index_temperature_profiles},
        'Density profile': {'plot_style':style_density_profiles,'key':'density','xlabel':'Density (kg/m3)','data':sp.density_profiles,'index':index_density_profiles},
        'LWC': {'plot_style':style_lwc_profiles,'key':'lwc','xlabel':'LWC (%)','data':sp.lwc_profiles,'index':index_lwc_profiles},
        'Strength profile': {'plot_style':style_strength_profiles,'key':'strength','xlabel':'Strength (N)','data':sp.strength_profiles,'index':index_strength_profiles},
        'SSA profile': {'plot_style':style_ssa_profiles,'key':'ssa','xlabel':'SSA (m2/kg)','data':sp.ssa_profiles,'index':index_ssa_profiles},
        'Impurity profile': {'plot_style':style_impurity_profiles,'key':'mass_fraction','xlabel':'Impurity mass fraction','data':sp.impurity_profiles,'index':index_impurity_profiles},
        'Other scalar profile': {'plot_style':'point','key':'data','xlabel':'Other scalar variable','data':sp.other_scalar_profiles,'index':index_scalar_profiles},
    }
    step_profiles_key_list = ['hardness']

    n_to_plot = 0
    for v in to_plot.values():
        if v['data'] is not None and (not isinstance(v['data'], list) or (len(v['data'])>0 and v['index'] is not None)):
            v['ok'] = True; n_to_plot += 1
        else:
            v['ok'] = False
    if n_to_plot == 0:
        raise ValueError("Nothing to plot.")

    fig, axs = plt.subplots(nrows=(n_to_plot - 1)//4 + 1,
                            ncols=min(n_to_plot, 4),
                            figsize=(16,15), sharey=True,
                            gridspec_kw={'wspace':0.4},
                            squeeze=False)  # squeeze=False prevents single-Axes issue
    axes = axs.ravel().tolist()

    n = 0
    for v in to_plot.values():
        if not v['ok']: 
            continue
        common = {'ylabel': 'Height (m)' if n % 4 == 0 else None}
        if v['plot_style'] == 'profile':
            plot_utils.plot_strati_profile(axes[n], v['data'], xlabel=v['xlabel'], **common, **kwargs)
        elif v['plot_style'] == 'point':
            plot_utils.plot_point_profile(axes[n], v['data'], v['key'], v['index'], xlabel=v['xlabel'], **common, **kwargs)
        elif v['plot_style'] == 'step':
            if v['key'] in step_profiles_key_list:
                plot_utils.plot_step_profile(axes[n], v['data'], v['key'], v['index'], xlabel=v['xlabel'], **common, **kwargs)
            else:
                plot_utils.plot_vline_profile(axes[n], v['data'], v['key'], v['index'], xlabel=v['xlabel'], **common, **kwargs)
        else:
            raise ValueError(f"Unknown style {v['plot_style']}")
        n += 1
    return fig

print("✓ plot_full_safe helper function defined")



In [ ]:
# Plot using built-in functions

# Quick overview plot
print("Plotting simple overview...")
try:
    spplot.plot_simple(sp)
    plt.show()
except Exception as e:
    print(f"⚠ Error in plot_simple: {e}")

# Full plot using safe wrapper
print("\nPlotting full profile (safe wrapper)...")
try:
    fig = plot_full_safe(sp)
    plt.show()
except Exception as e:
    print(f"⚠ Error in plot_full_safe: {e}")
    print("This may happen if no profiles are available to plot.")

# Custom stratigraphy panel
if hasattr(sp, 'stratigraphy_profile') and sp.stratigraphy_profile is not None:
    print("\nCustom stratigraphy panel...")
    fig, ax = plt.subplots(figsize=(5, 6))
    try:
        sppu.plot_strati_profile(ax, sp.stratigraphy_profile, 
                                xlabel="Hand hardness", 
                                ylabel="Height (m)", 
                                use_hardness=True)
        plt.show()
    except Exception as e:
        print(f"⚠ Error plotting stratigraphy: {e}")
else:
    print("⚠ No stratigraphy profile available for custom plot")



In [ ]:
# Export back to CAAML

# Create outputs directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Export to CAAML 6.0.5 (default)
out_path = output_dir / "profile_out.caaml"
try:
    spio.write_caaml6_xml(sp, str(out_path), version="6.0.5", indent=True)
    print(f"✓ Exported to CAAML 6.0.5: {out_path}")
except Exception as e:
    print(f"⚠ Error exporting to 6.0.5: {e}")

# Also export to CAAML 6.0.6 (writer supports both versions)
out_path_606 = output_dir / "profile_out_606.caaml"
try:
    spio.write_caaml6_xml(sp, str(out_path_606), version="6.0.6", indent=True)
    print(f"✓ Exported to CAAML 6.0.6: {out_path_606}")
except Exception as e:
    print(f"⚠ Error exporting to 6.0.6: {e}")

# Download link for Colab
if IN_COLAB:
    from google.colab import files
    print("\nDownload options:")
    if out_path.exists():
        print(f"  Downloading {out_path.name}...")
        files.download(str(out_path))
    if out_path_606.exists():
        print(f"  Downloading {out_path_606.name}...")
        files.download(str(out_path_606))

print("\nNote: Some user-defined data may not round-trip perfectly. The writer preserves core profile data but may lose custom metadata.")



In [ ]:
# JSON / dict views

from snowprofile.io import to_dict, to_json

# Convert to dictionary
sp_dict = to_dict(sp)
print("Top-level keys in dictionary representation:")
print(list(sp_dict.keys())[:10])  # Show first 10 keys
if len(sp_dict) > 10:
    print(f"... and {len(sp_dict) - 10} more keys")

# Convert to JSON
sp_json = to_json(sp)
print(f"\nJSON representation length: {len(sp_json)} characters")
print(f"JSON preview (first 200 chars):\n{sp_json[:200]}...")



## (Optional) Merge Profiles

If you have a second CAAML file, you can merge profiles using `SnowProfile.merge()`. This is useful for combining observations from different times or locations.

Example code (commented out - uncomment if you have a second file):

```python
# Load second profile
# sp2 = spio.read_caaml6_xml(str(CAAML_PATH_2))
# 
# # Merge profiles
# sp_merged = sp.merge(sp2)
# 
# # Export merged profile
# spio.write_caaml6_xml(sp_merged, "outputs/merged_profile.caaml", version="6.0.5")
```


## Try-It Exercises

Here are some exercises to practice with the `snowprofile` package:

1. **Change temperature profile index**: Modify the temperature gradient analysis to use a different temperature profile (e.g., `temp_profiles[1]` if available)

2. **Import density CSV**: Use `snowprofile.io.profile_csv.read_csv_profile()` to import a density profile from CSV and merge it with your CAAML profile. See the I/O CSV documentation section for details.

3. **Compare CAAML versions**: Export the same profile as both 6.0.5 and 6.0.6, then compare the XML files to see differences.

4. **Compute SWE from density**: Use a density profile to compute Snow Water Equivalent (SWE) by integrating density × thickness:
   ```python
   # SWE = ∫ density(z) dz
   # Approximate as sum of density × thickness for each layer
   ```

5. **Custom visualization**: Create your own plot combining multiple profiles (e.g., temperature and density on the same axes with different scales)

6. **Filter layers**: Extract and analyze only specific layers (e.g., surface layers, weak layers) based on hardness or other criteria


## Troubleshooting

### CAAML Reader Notes

- **Version support**: Reader supports CAAML versions up to 6.0.5
- **Namespace requirement**: Files must use a single, consistent CAAML namespace
- **Missing data**: Missing numeric values become `None` in the SnowProfile object
- **Category values**: Category values (like hand hardness) are converted to numeric mid-classes
- **Data loss**: Some custom/user-defined data may not round-trip when reading and rewriting

### Common Issues

1. **`plot_full` single-Axes error**: 
   - **Problem**: `TypeError: 'Axes' object is not iterable` when only one subplot is created
   - **Solution**: Use `plot_full_safe()` wrapper provided in this notebook, which uses `squeeze=False` and `axs.ravel()` to handle edge cases

2. **`loc` attribute validation errors**:
   - **Problem**: `ValueError: Unauthorized value for key loc`
   - **Cause**: Some CAAML exporters use non-standard `loc` values like "Surface", "Bottom" instead of "T", "B"
   - **Solution**: Use the `sanitize_caaml_loc()` helper function to normalize values before reading

3. **Missing temperature profiles**:
   - **Problem**: Code fails when trying to analyze temperature gradients
   - **Solution**: Always check if profiles exist before accessing them: `if sp.temperature_profiles: ...`

4. **Hand-hardness confusion**:
   - **Note**: Hand-hardness values appear in the stratigraphy panel. The separate Hardness profile subplot requires actual `HardnessProfile` data (numeric hardness measurements), not hand-hardness categories. See the plot functions documentation for details.

### Getting Help

- Check the `snowprofile` package documentation for function names and signatures
- Reference the official CAAML schema documentation
- Use `help(snowprofile.io.read_caaml6_xml)` for function documentation
